In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
#change the file name to correspond to your text file location
data = np.genfromtxt('./Group Project/ASTR19_F24_group_project_data.txt', dtype=[('day', 'i8'), ('time', 'U6'), ('height', 'f8')])

#the
day = []
time = []
y = []

for i in range(82):
    #assign first, second and third columns to day, time, and height
    day.append(data[i][0])
    time.append(data[i][1])
    y.append(data[i][2])

print(day)
print('')
print(time)
print('')
print(y)

In [ ]:
percents = []
for times in time:
    hourmin = times.split(':')
    percent = (float(hourmin[0]) + float(hourmin[1])/60) /24
    percents.append(percent)
x = [percent + daynum for percent, daynum in zip(percents, day)]

In [ ]:
f = plt.figure(figsize=(7,7))
plt.errorbar(x, y, yerr=np.full(len(y), 0.25), fmt='o')
plt.xlabel('Day')
plt.ylabel('Height of tide (ft)')
plt.show()

In [ ]:
#defining the ocillating model
def o_model(x, A1, f1, p1, A2, f2, p2, C):
    return (A1 * np.sin(f1 * x + p1)) + (A2 * np.sin(f2 * x + p2)) + C

In [ ]:
x = np.array(x, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# Amplitudes
A1_init = ((max(y) - min(y)) / 2)
A1_fit = tf.Variable(A1_init, dtype=tf.float32)

A2_init = A1_init / 2
A2_fit = tf.Variable(A2_init, dtype=tf.float32)

# Periods

f1_init = 2*np.pi
f1_fit = tf.Variable(f1_init, dtype=tf.float32)

f2_init = np.pi/15
f2_fit = tf.Variable(f2_init, dtype=tf.float32)

# Phases

p1_init = 0
p1_fit = tf.Variable(p1_init, dtype=tf.float32)

p2_init = 2
p2_fit = tf.Variable(p2_init, dtype=tf.float32)

# Offset

C_init = np.mean(y)
C_fit = tf.Variable(C_init, dtype=tf.float32)

In [ ]:
@tf.function
def f_tide(x):
    """A more complex tidal model function."""
    return (A1_fit * tf.math.sin(f1_fit * x + p1_fit)) + (A2_fit * tf.math.sin(f2_fit * x + p2_fit)) + C_fit

In [ ]:
@tf.function
def loss_function(y_true, y_pred):
    return tf.keras.losses.MeanSquaredError()(y_true, y_pred)

In [ ]:
# Define the optimizer (Adam optimizer)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
# Training loop
epochs = 5000
display_step = 1000

for epoch in range(epochs):
    if ((epoch % display_step) == 0):
        pred = f_tide(x)
        loss = loss_function(pred, y)
        print(f"Epoch {epoch} | Loss = {loss.numpy()}, A1: {A1_fit.numpy()}, f1: {f1_fit.numpy()}, p1: {p1_fit.numpy()}, A2: {A2_fit.numpy()}, f2: {f2_fit.numpy()}, p2: {p2_fit.numpy()}, C: {C_fit.numpy()}")

    with tf.GradientTape() as tape:
        pred = f_tide(x)
        loss = loss_function(pred, y)

    gradients = tape.gradient(loss, [A1_fit, f1_fit, p1_fit, A2_fit, f2_fit, p2_fit, C_fit])
    optimizer.apply_gradients(zip(gradients, [A1_fit, f1_fit, p1_fit, A2_fit, f2_fit, p2_fit, C_fit]))


print(f"Epoch {epoch} | Loss = {loss.numpy()}, A1: {A1_fit.numpy()}, f1: {f1_fit.numpy()}, p1: {p1_fit.numpy()}, A2: {A2_fit.numpy()}, f2: {f2_fit.numpy()}, p2: {p2_fit.numpy()}, C: {C_fit.numpy()}")
print('Done!')

In [ ]:
x_model = np.sort(x)
y_model = o_model(x_model, A1_fit, f1_fit, p1_fit, A2_fit, f2_fit, p2_fit, C_fit)

In [ ]:
#plotting the 
f,ax = plt.subplots(1,1,figsize=(7,7))
ax.errorbar(x, y, yerr=np.full(len(x), 0.25), fmt='o', label='Data')
ax.plot(x_model, y_model, color='orange',label='Best-fit Model')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.legend(frameon=False,handletextpad=0)

In [ ]:
x = np.asarray(np.random.uniform(low=min(x), high=max(x), size=100), dtype=np.float32)
y = np.asarray(o_model(x, A1_fit, f1_fit, p1_fit, A2_fit, f2_fit, p2_fit, C_fit) + 0.25*np.random.randn(len(x)), dtype=np.float32)
y_err = np.full(100, 0.25, dtype=np.float32)

idx_model = np.argsort(x)
y_data = y[idx_model]
y_model = o_model(x[idx_model], A1_fit, f1_fit, p1_fit, A2_fit, f2_fit, p2_fit, C_fit)

residuals = y_data - y_model
residuals = residuals.numpy()

print(residuals)

f, ax = plt.subplots(1, 1, figsize=(7, 7))

for tick in ax.xaxis.get_ticklabels():
    tick.set_fontsize(14)
for tick in ax.yaxis.get_ticklabels():
    tick.set_fontsize(14)


ax.errorbar(x, residuals, y_err, fmt='o', label='Residuals')
ax.set_ylim([-2, 2])
ax.set_xlabel('x', fontsize=14)
ax.set_ylabel('y', fontsize=14)
plt.legend(frameon=False, fontsize=14, handletextpad=0)
plt.show()

In [ ]:
residual_mean = np.mean(residuals)
residual_std = np.std(residuals)

In [ ]:
def gaussian(x, mu, s):
    return 1./(2.*np.pi*s**2)**0.5 * np.exp(-0.5*((x-mu)/s)**2)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))
for tick in ax.xaxis.get_ticklabels():
    tick.set_fontsize(14)
for tick in ax.yaxis.get_ticklabels():
    tick.set_fontsize(14)

ax.hist(residuals, bins=10, range=(-0.75,0.75), alpha=0.5, edgecolor='white', density=True)

x_g = np.linspace(-5*residual_std, 5*residual_std, 1000)
ax.plot(x_g, gaussian(x_g, residual_mean, residual_std), color='red')
ax.set_xlim([-2, 2])
ax.set_xlabel('x', fontsize=14)
ax.set_ylabel('y', fontsize=14)
plt.show()

In [ ]:
residuals = np.append(residuals, 2.)
print(residuals)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))
for tick in ax.xaxis.get_ticklabels():
    tick.set_fontsize(14)
for tick in ax.yaxis.get_ticklabels():
    tick.set_fontsize(14)

ax.hist(residuals, bins=30, range=(-2.25, 2.25), alpha=0.5, edgecolor='white', density=True)

x_g = np.linspace(-5*residual_std, 5*residual_std, 1000)
ax.plot(x_g, gaussian(x_g, residual_mean, residual_std), color='red')
ax.set_xlim([-2.2, 2.2])
ax.set_xlabel('x', fontsize=14)
ax.set_ylabel('y', fontsize=14)
plt.show()

In [ ]:
residual_std = np.std(residuals)
print(f"The standard deviation of the residuals is {residual_std}")

In [ ]:
print(f"The outlier is about {(2. - np.mean(residuals)) / np.std(residuals)} standard deviations away from the mean.")